In [1]:
using XGPaint, Cosmology

filename = "/home/zack/scratch/websky_halos-light.hdf5"

# catalogs are stored in comoving (x,y,z) and M/Msun
halo_pos, halo_mass = read_halo_catalog_hdf5(filename);

In [9]:
"""
Fill in basic halo properties.
"""
function get_angles_and_redshift(halo_pos::Array{T,2}, cosmo::Cosmology.FlatLCDM{T};
        min_redshift=0.0, max_redshift=4.5) where T
    N_halos = size(halo_pos, 2)
    θ = Array{T}(undef, N_halos)
    ϕ = Array{T}(undef, N_halos)
    redshift = Array{T}(undef, N_halos)
    dist = Array{T}(undef, N_halos)

    r2z = XGPaint.build_r2z_interpolator(min_redshift, max_redshift, cosmo)
    Threads.@threads :static for i in 1:N_halos
        dist[i] = sqrt(halo_pos[1,i]^2 + halo_pos[2,i]^2 + halo_pos[3,i]^2)
        redshift[i] = r2z(dist[i])
        θ[i], ϕ[i] = XGPaint.Healpix.vec2ang(halo_pos[1,i], halo_pos[2,i], halo_pos[3,i])
    end

    return dist, redshift, θ, ϕ
end

get_angles_and_redshift

In [10]:
cosmo = XGPaint.get_cosmology(h=0.677f0, OmegaM=0.310f0)

dist, redshift, theta, phi = get_angles_and_redshift(halo_pos, cosmo)

(Float32[7671.1914, 7674.854, 7625.1035, 7604.1143, 7322.292, 7508.435, 7367.987, 7331.093, 7404.6294, 7413.0615  …  6475.194, 6403.712, 6400.1826, 6431.2793, 6412.8545, 6548.201, 6520.3726, 6531.156, 6633.721, 6671.001], Float32[4.5124907, 4.518473, 4.437719, 4.4041524, 3.9816225, 4.2549567, 4.0467434, 3.994068, 4.099875, 4.1122184  …  2.967984, 2.8965933, 2.893117, 2.9239035, 2.90562, 3.0428712, 3.014086, 3.0252044, 3.1332307, 3.1735413], Float32[2.208283, 2.2011456, 2.1996765, 2.1911328, 1.6400945, 1.6956769, 1.6977051, 1.6904302, 1.6849878, 1.6831716  …  1.26901, 1.2655013, 1.2653497, 1.2668978, 1.2657248, 1.2726425, 1.271406, 1.2721376, 1.2768482, 1.2785295], Float32[3.9280708, 3.9336264, 3.9232962, 3.936465, 3.2286189, 3.257981, 3.210661, 3.208916, 3.2096262, 3.2200599  …  1.1146858, 1.1412125, 1.1433005, 1.1450206, 1.1652906, 1.1201836, 1.1379812, 1.1362743, 1.1259247, 1.127944])